In [1]:
from bs4 import BeautifulSoup as Soup
import urllib.request
import requests
import numpy as np
import pandas as pd
import re

In [2]:
url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup= Soup(page, "html.parser")

In [3]:
#https://stackoverflow.com/questions/33766740/beautifulsoup-find-table-with-specified-class-on-wikipedia-page

tab = soup.find("table",{"class":"wikitable sortable"})
columns= [i.getText().split('\n')[0] for i in tab.findAll("th")]
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


In [4]:
alldata= np.array([i.getText().split('\n')[0] for i in tab.findAll("td")])
rows= alldata.reshape(287, 3)

In [5]:
df= pd.DataFrame(data= rows, columns= columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Stage 2: Geocoding

In [6]:
!wget https://cocl.us/Geospatial_data

--2020-01-31 21:17:42--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 119.81.168.76, 161.202.50.39, 119.81.168.75
Connecting to cocl.us (cocl.us)|119.81.168.76|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-31 21:17:45--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-31 21:17:46--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv

In [11]:
geodf= pd.read_csv('Geospatial_data')

In [12]:
geodf

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [15]:
newdf= pd.merge(df, geodf, left_on='Postcode', right_on='Postal Code').rename(columns = {'Postcode':'PostalCode'}).drop(['Postal Code'], axis= 1) 

In [16]:
newdf

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
206,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
207,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
208,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999
